In [1]:
import torch
import numpy as np
import os
from torch.utils import data
import torch.nn as nn
import pandas as pd

In [2]:
!pip install cntext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.1/632.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

# 数据预处理

In [3]:
df = pd.read_json("../input/data-text-mining/news2016zh_valid.json", lines=True)
df.head() # len(df)=76797 

,news_id,keywords,desc,title,source,time,content
0,470308283,借鉴：这篇最受欢迎校训，没有一个字讲学习,“我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的；我是为...,借鉴：这篇最受欢迎校训，没有一个字讲学习,龙乡教育,10-14 21:55,“我知道，我不是因为偶然才来到这个世界，我是为了践行一个平凡、美丽、无私的梦想而来的；我是为...
1,410240504,市民淘宝购物一年四度被忽悠 买货容易退货难,张女士提供豆皮有虫尸的证据照片。受访者供图淘宝上购物，一年被忽悠四次。这样的事情就被市民张女...,市民淘宝购物一年四度被忽悠 买货容易退货难,主流媒体-媒体平台入驻自媒体,03-24 14:13,张女士提供豆皮有虫尸的证据照片。受访者供图淘宝上购物，一年被忽悠四次。这样的事情就被市民张...
2,410933104,[统计]机械.电器类公司市场表现(20150407),@名称代码收盘价日涨跌周涨跌月涨跌季涨跌深华新0000100.0000.00%0.00%0....,[统计]机械.电器类公司市场表现(20150407),移动中金在线,04-08 06:40,@名称代码收盘价日涨跌周涨跌月涨跌季涨跌深华新0000100.0000.00%0.00%0....
3,470124533,[每日一技]四维绕杆跑练习,爱高体育，有趣兼有料的校园足球资讯平台。每日为你分享一招足球技巧！让你更懂得如何运用爱高安全...,[每日一技]四维绕杆跑练习,爱高体育mp,10-12 22:15,爱高体育，有趣兼有料的校园足球资讯平台。每日为你分享一招足球技巧！让你更懂得如何运用爱高安...
4,471705751,【漫展活动】风云小分队在YACA,大家期待已久的yaca万圣动漫音乐节终于来啦，风云动漫社の动物园小分队早早来到了展会现场布置...,【漫展活动】风云小分队在YACA,顺职风云动漫社,10-29 01:07,大家期待已久的yaca万圣动漫音乐节终于来啦，风云动漫社の动物园小分队早早来到了展会现场布...


In [4]:
from sklearn.model_selection import train_test_split
x, y = df["keywords"].values, df["title"].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x_train[:3], y_train[:3]

(array(['什么样的医改才能让钟南山院士满意', '七旬老人，外籍女子，长城', '卖盘飙升致压制大盘反弹 等待上攻机会'],
       dtype=object),
 array(['什么样的医改才能让钟南山院士满意', '北京：外籍女子在长城上撞死七旬老人', '卖盘飙升致压制大盘反弹 等待上攻机会'],
       dtype=object))

In [5]:
def read_keywords_title(data_dir):
    """读取keywords_title文本序列和目标文本序列"""
    df = pd.read_json(data_dir, lines=True)
    x, y = df["keywords"].values, df["title"].values
    return (x.astype(str).tolist(), y.astype(str).tolist())

In [6]:
data_dir = "../input/data-text-mining/news2016zh_valid.json"

k2t_data = read_keywords_title(data_dir)

for x, y in zip(k2t_data[0][:3], k2t_data[1][:3]):
    print('title：', y, 'keywords:', x[0:60])

title： 借鉴：这篇最受欢迎校训，没有一个字讲学习 keywords: 借鉴：这篇最受欢迎校训，没有一个字讲学习
title： 市民淘宝购物一年四度被忽悠 买货容易退货难 keywords: 市民淘宝购物一年四度被忽悠 买货容易退货难
title： [统计]机械.电器类公司市场表现(20150407) keywords: [统计]机械.电器类公司市场表现(20150407)


In [7]:
import re
import jieba
import cntext as ct

stopwords = ct.load_pkl_dict('STOPWORDS.pkl')['STOPWORDS']['chinese']

def tokenize(lines):
    tokenized_list = []
    for line in lines:
        words = jieba.lcut(line, cut_all=False)
        words = [w for w in words if w not in stopwords]
        tokenized_list.append(words)
    return tokenized_list

In [8]:
import collections
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

def truncate_pad(line, num_steps, padding_token):
    """截取或者填充token序列"""
    if len(line) > num_steps:
        return line[:num_steps]  # Truncate
    return line + [padding_token] * (num_steps - len(line))  # Pad

In [9]:
def turn_type(x, dtype):
    return x.type(dtype)

def get_sum(x, add):
    return x.sum(add)

def build_array_text(lines, vocab, num_steps):
    """Transform text sequences of machine translation into minibatches."""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([
        truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = get_sum(turn_type(array != vocab['<pad>'], torch.int32), 1)
    return array, valid_len

In [10]:
def tensor2loader(data_arrays, batch_size, shuffle=True):
    """构建torch.dataloader"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=shuffle)

In [11]:
num_steps = 15 # time_step=20
batch_size = 64

source = tokenize(k2t_data[1])
target = tokenize(k2t_data[0])

src_vocab = Vocab(source, min_freq=5, reserved_tokens=['<pad>', '<bos>', '<eos>'])
tgt_vocab = Vocab(target, min_freq=5, reserved_tokens=['<pad>', '<bos>', '<eos>'])

src_array, src_valid_len = build_array_text(source, src_vocab, num_steps)
tgt_array, tgt_valid_len = build_array_text(target, tgt_vocab, num_steps)

data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
data_iter = tensor2loader(data_arrays, batch_size)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.995 seconds.
Prefix dict has been built successfully.


In [12]:
src_array.shape, src_valid_len.shape, tgt_array.shape, tgt_valid_len.shape

(torch.Size([76797, 15]),
 torch.Size([76797]),
 torch.Size([76797, 15]),
 torch.Size([76797]))

In [13]:
#@save
class Seq2SeqEncoder(nn.Module):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)

    def forward(self, X, *args):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state[0]的形状:(num_layers,batch_size,num_hiddens)
        return output, state


In [14]:
class Seq2SeqDecoder(nn.Module):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        context = state[-1].repeat(X.shape[0], 1, 1)
        X_and_context = torch.cat((X, context), 2)
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state[0]的形状:(num_layers,batch_size,num_hiddens)
        return output, state

In [15]:
class EncoderDecoder(nn.Module):
    """The base class for the encoder-decoder architecture."""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)

In [16]:
#@save
def sequence_mask(X, valid_len, value=0):
    """在序列中屏蔽不相关的项"""
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

In [17]:
#@save
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """带遮蔽的softmax交叉熵损失函数"""
    # pred的形状：(batch_size,num_steps,vocab_size)
    # label的形状：(batch_size,num_steps)
    # valid_len的形状：(batch_size,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction='none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss

In [18]:
#@save
def predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps,
                    device):
    """序列到序列模型的预测
       src_sentense是以_!_为分隔符的句子
    """
    # 在预测时将net设置为评估模式
    net.eval()
    src_tokens = src_vocab[src_sentence.split('_!_')] + [
        src_vocab['<eos>']]
    enc_valid_len = torch.tensor([len(src_tokens)], device=device)
    src_tokens = truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    
    # 添加批量轴
    enc_X = torch.unsqueeze(
        torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    # 添加批量轴
    dec_X = torch.unsqueeze(torch.tensor(
        [tgt_vocab['<bos>']], dtype=torch.long, device=device), dim=0)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # 我们使用具有预测最高可能性的词元，作为解码器在下一时间步的输入
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        # 一旦序列结束词元被预测，输出序列的生成就完成了
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)), attention_weight_seq

In [19]:
src_sentence = '市民_!_ 淘宝_!_购物_!_一年_!_ 四度_!_ 被_!_ 忽悠_!_ _!_买货_!_容易_!_退货_!_难'

In [20]:
from tqdm.auto import tqdm

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_size, num_hiddens, num_layers, dropout = 50, 256, 2, 0.1
encoder = Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers,
                        dropout)
decoder = Seq2SeqDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers,
                        dropout)
net = EncoderDecoder(encoder, decoder)
net.to(device)
lr, num_epochs = 5e-3, 50
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = MaskedSoftmaxCELoss()
print("尝试预测的kewords:{}".format(src_sentence))
for epoch in tqdm(range(num_epochs)):
    net.train()
    print("||epoch:{}||".format(epoch))
    epoch_loss = 0.
    for batch in data_iter:
        optimizer.zero_grad()
        X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]
        bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0],
                      device=device).reshape(-1, 1)
        dec_input = torch.cat([bos, Y[:, :-1]], 1)  # 强制教学
        Y_hat, _ = net(X, dec_input, X_valid_len)
        l = loss(Y_hat, Y, Y_valid_len)
        l.sum().backward()      # 损失函数的标量进行“反向传播”
        num_tokens = Y_valid_len.sum()
        optimizer.step()
        with torch.no_grad():
            epoch_loss += l.sum() / X.size(0)
    epoch_loss = epoch_loss / len(data_iter)
    epoch_title, _ = predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps, device)
    print("||epoch_loss:{}||预测的title:{}".format(epoch_loss, epoch_title))

尝试预测的kewords:市民_!_ 淘宝_!_购物_!_一年_!_ 四度_!_ 被_!_ 忽悠_!_ _!_买货_!_容易_!_退货_!_难


  0%|          | 0/50 [00:00<?, ?it/s]

||epoch:0||
||epoch_loss:3.339498281478882||预测的title:市民 <unk> <unk> <unk> <unk> <unk>   <unk> <unk>
||epoch:1||
||epoch_loss:2.277987241744995||预测的title:<unk> <unk>
||epoch:2||
||epoch_loss:1.6775989532470703||预测的title:市民 市民 <unk> <unk> <unk> <unk> <unk>   <unk> <unk> <unk> <unk> 难 <unk> 难
||epoch:3||
||epoch_loss:1.3784126043319702||预测的title:<unk> <unk>
||epoch:4||
||epoch_loss:1.22378671169281||预测的title:市民 <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
||epoch:5||
||epoch_loss:1.125881314277649||预测的title:购物 购物
||epoch:6||
||epoch_loss:1.0609477758407593||预测的title:市民 <unk> <unk> <unk> <unk> <unk> <unk>
||epoch:7||
||epoch_loss:1.0210938453674316||预测的title:喝酒 <unk> <unk> 一年 <unk> <unk> <unk>   <unk> 难 <unk> 难 ; 花样 退货
||epoch:8||
||epoch_loss:0.9652506709098816||预测的title:市民 <unk> <unk> <unk> <unk> <unk>   <unk> 购物 基因 奶茶 难 击败 馆 功效
||epoch:9||
||epoch_loss:0.9325369596481323||预测的title:市民 <unk> <unk> <unk> <unk> <unk>   一年 退货 女朋友 移动 移动 S7 移动 移动
||epoch: